# Business Problem

For this project, a client has request the production of a MySQL database on Movies from a subset of IMDB's publicly available dataset. They would like an analysis of this database to help determine what makes a movie successful and recommendations on how to make a successful movie.

## Part 1: 

Download several files from IMDB’s movie data set and filter out the subset of moves requested by the stakeholder.

## Specifications

- Exclude any movie with missing values for genre or runtime
- Include only full-length movies (titleType = "movie").
- Include only fictional movies (not from documentary genre)
- Include only movies that were released 2000 - 2021 (include 2000 and 2021)
- Include only movies that were released in the United States

# Data Location
- The dataset files can be accessed and downloaded from https://datasets.imdbws.com/. 
- Data Source: https://www.themoviedb.org/about/logos-attribution

# Data Dictionary

## Title Aka 

- titleId (string) - a tconst, an alphanumeric unique identifier of the title
- ordering (integer) – a number to uniquely identify rows for a given titleId
- title (string) – the localized title
- region (string) - the region for this version of the title
- language (string) - the language of the title
- types (array) - Enumerated set of attributes for this alternative title. One or more of the following: "alternative",  "dvd", "festival", "tv", "video", "working", "original", "imdbDisplay". New values may be added in the future without warning
- attributes (array) - Additional terms to describe this alternative title, not enumerated
- isOriginalTitle (boolean) – 0: not original title; 1: original title

## Title Basics 

- tconst (string) - alphanumeric unique identifier of the title
- titleType (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
- primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release
- originalTitle (string) - original title, in the original language
- isAdult (boolean) - 0: non-adult title; 1: adult title
- startYear (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year
- endYear (YYYY) – TV Series end year. ‘\N’ for all other title types
- runtimeMinutes – primary runtime of the title, in minutes
- genres (string array) – includes up to three genres associated with the title

## Title Ratings

- tconst (string) - alphanumeric unique identifier of the title
- averageRating – weighted average of all the individual user ratings
- numVotes - number of votes the title has received

# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os

# Download Data

In [2]:
#basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
#aka_url= "https://datasets.imdbws.com/title.akas.tsv.gz"
#ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

In [3]:
# Create Dataframes
#basics_df = pd.read_csv(basics_url, sep='\t', low_memory=False)

#aka_df= basics = pd.read_csv(aka_url, sep='\t', low_memory=False)

#ratings_df = pd.read_csv(ratings_url, sep='\t', low_memory=False)

In [4]:
# making new folder with os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

['title_akas.csv.gz', 'title_basics.csv.gz', 'title_ratings.csv.gz']

In [5]:
# Open saved file and preview again
basics_df = pd.read_csv("Data/title_basics.csv.gz", low_memory=False)
basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,118,"Comedy,Fantasy,Romance"
1,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013.0,120,"Drama,History"
2,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,70,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,100,"Comedy,Horror,Sci-Fi"


In [6]:
# Create dataframe
#akas_df = pd.read_csv(aka_url,sep='\t', low_memory=False)

In [7]:
# Open saved file and preview again
akas_df = pd.read_csv("Data/title_akas.csv.gz", low_memory=False)
akas_df.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [8]:
# Create dataframe
#ratings_df = pd.read_csv(ratings_url,sep='\t', low_memory=False)

In [9]:
# Open saved file and preview again
ratings_df = pd.read_csv("Data/title_ratings.csv.gz", low_memory=False)
ratings_df.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1982
1,tt0000002,5.8,265
2,tt0000003,6.5,1838
3,tt0000004,5.5,178
4,tt0000005,6.2,2625


# Preprocessing

## Title Basics 

- Replace "\N" with np.nan
- Eliminate movies that are null for runtimeMinutes
- Eliminate movies that are null for genre
- keep only titleType==Movie
- keep startYear 2000-2022
- Eliminate movies that include "Documentary" in genre (see tip below)
- Keep only US movies (Use AKAs table, see "Filtering one dataframe based on another" section below)

In [10]:
basics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81803 entries, 0 to 81802
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          81803 non-null  object 
 1   titleType       81803 non-null  object 
 2   primaryTitle    81803 non-null  object 
 3   originalTitle   81803 non-null  object 
 4   isAdult         81803 non-null  int64  
 5   startYear       81803 non-null  float64
 6   runtimeMinutes  81803 non-null  int64  
 7   genres          81803 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 5.0+ MB


In [11]:
basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,118,"Comedy,Fantasy,Romance"
1,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013.0,120,"Drama,History"
2,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,70,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,100,"Comedy,Horror,Sci-Fi"


In [12]:
basics_df['startYear'] = basics_df['startYear'].astype(float)

In [13]:
# Replace "\N" with np.nan
basics_df.replace({'\\N':np.nan},inplace=True)

In [14]:
# Eliminate movies that are missing values for runtimeMinutes, genres, startYear
basics_df = basics_df.dropna(subset = ['runtimeMinutes', 'genres'])

In [15]:
# Keep titleType movie
basics_df = basics_df.loc[basics_df['titleType'] == 'movie']

In [16]:
# Keep startYear 2000-2021

basics_df = basics_df.loc[(basics_df['startYear'] >= 2000) & (basics_df['startYear'] <=2021)]

In [17]:
basics_df['startYear'].value_counts()

2019.0    5877
2018.0    5780
2017.0    5643
2016.0    5255
2021.0    5154
2015.0    5053
2020.0    5005
2014.0    4911
2013.0    4708
2012.0    4519
2011.0    4226
2010.0    3859
2009.0    3552
2008.0    2908
2007.0    2573
2006.0    2433
2005.0    2179
2004.0    1900
2003.0    1679
2001.0    1570
2002.0    1567
2000.0    1452
Name: startYear, dtype: int64

In [18]:
# Eliminate movies that include "Documentary" in genre
documentary_filter = basics_df['genres'].str.contains('documentary', case=False)
basics_df = basics_df[~documentary_filter]

In [19]:
basics_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81803 entries, 0 to 81802
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          81803 non-null  object 
 1   titleType       81803 non-null  object 
 2   primaryTitle    81803 non-null  object 
 3   originalTitle   81803 non-null  object 
 4   isAdult         81803 non-null  int64  
 5   startYear       81803 non-null  float64
 6   runtimeMinutes  81803 non-null  int64  
 7   genres          81803 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 5.6+ MB


## Title AKAs
- Keep only US movies.
- Replace "\N" with np.nan

In [20]:
# Replace "\N" with np.nan
akas_df.replace({'\\N':np.nan},inplace=True)

In [21]:
#akas_df = akas_df.loc[akas_df['region'] =='US']

In [22]:
us_only = akas_df['region'] == 'US'

In [23]:
akas_df = akas_df[us_only]
akas_df.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [24]:
akas_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1447400 entries, 0 to 1447399
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   titleId          1447400 non-null  object 
 1   ordering         1447400 non-null  int64  
 2   title            1447400 non-null  object 
 3   region           1447400 non-null  object 
 4   language         3964 non-null     object 
 5   types            980572 non-null   object 
 6   attributes       46856 non-null    object 
 7   isOriginalTitle  1446058 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 99.4+ MB


In [25]:
# Filter the basics table to only include US by using the filter aka dataframe
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics_df['tconst'].isin(akas_df['titleId'])
keepers

0        True
1        True
2        True
3        True
4        True
         ... 
81798    True
81799    True
81800    True
81801    True
81802    True
Name: tconst, Length: 81803, dtype: bool

In [26]:
basics_df = basics_df[keepers]
basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,118,"Comedy,Fantasy,Romance"
1,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013.0,120,"Drama,History"
2,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,70,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,100,"Comedy,Horror,Sci-Fi"


## Title Ratings
- Replace "\N" with np.nan (if any)
- Keep only US movies (Use AKAs table, see "Filtering one dataframe based on another" section below)

In [27]:
# Replace "\N" with np.nan
ratings_df.replace({'\\N':np.nan},inplace=True)

In [28]:
ratings_df.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1982
1,tt0000002,5.8,265
2,tt0000003,6.5,1838
3,tt0000004,5.5,178
4,tt0000005,6.2,2625


In [29]:
ratings_us = ratings_df['tconst'].isin(akas_df['titleId'])

In [30]:
ratings_df= ratings_df[ratings_us]
ratings_df

,tconst,averageRating,numVotes
0,tt0000001,5.7,1982
1,tt0000002,5.8,265
4,tt0000005,6.2,2625
5,tt0000006,5.1,182
6,tt0000007,5.4,820
...,...,...,...
1323868,tt9916200,8.1,230
1323869,tt9916204,8.2,263
1323876,tt9916348,8.3,18
1323877,tt9916362,6.4,5390


In [31]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 502108 entries, 0 to 1323882
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         502108 non-null  object 
 1   averageRating  502108 non-null  float64
 2   numVotes       502108 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.3+ MB


# df to csv's read in from Data file

In [32]:
## Save dataframe to file
basics_df.to_csv("Data/title_basics.csv.gz", compression='gzip', index=False)

In [33]:
## Save dataframe to file
ratings_df.to_csv("Data/title_ratings.csv.gz", compression='gzip', index=False)

In [34]:
## Save dataframe to file
akas_df.to_csv("Data/title_akas.csv.gz", compression='gzip', index=False)